In [14]:
import os
import numpy as np
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [15]:
class CustomDataset(Dataset):
    def __init__(self, folder_path):
        self.image_list = []
        self.labels = []
        for filename in os.listdir(folder_path):
            if 'auto' in filename:
                self.labels.append([1, 0, 0, 0, 0])
            elif 'moto' in filename:
                self.labels.append([0, 1, 0, 0, 0])
            elif 'casa' in filename:
                self.labels.append([0, 0, 1, 0, 0])
            elif 'edificio' in filename:
                self.labels.append([0, 0, 0, 1, 0])
            elif 'gato' in filename:
                self.labels.append([0, 0, 0, 0, 1])
            if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
                img_path = os.path.join(folder_path, filename)
                img = Image.open(img_path)
                img = img.resize((224, 224))
                img_array = np.array(img) / 255.0
                self.image_list.append(img_array)

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image = torch.tensor(self.image_list[idx], dtype=torch.float32).permute(2, 0, 1)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return image, label

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [17]:
class SimpleCNN(pl.LightningModule):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 128, kernel_size=3)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(128, 128, kernel_size=3)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.conv3 = nn.Conv2d(128, 128, kernel_size=3)
        self.pool3 = nn.MaxPool2d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 26 * 26, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 5)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.softmax(self.fc5(x))
        return x

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, torch.argmax(labels, dim=1))
        self.log('train_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, torch.argmax(labels, dim=1))
        predictions = torch.argmax(outputs, dim=1)
        acc = accuracy_score(torch.argmax(labels, dim=1).cpu().numpy(), predictions.cpu().numpy())
        self.log('test_loss', loss)
        self.log('test_accuracy', torch.tensor(acc))
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

In [18]:
train_dataset = CustomDataset('dataset/train')
test_dataset = CustomDataset('dataset/test')

In [19]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

In [20]:
model = SimpleCNN()

In [21]:
model.to(device)  # Mueve el modelo a la GPU

SimpleCNN(
  (conv1): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=86528, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=5, bias=True)
  (softmax): Softmax(dim=1)
)

In [25]:
trainer = Trainer(max_epochs=10)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [26]:
trainer.fit(model, train_loader)


   | Name    | Type      | Params | Mode 
-----------------------------------------------
0  | conv1   | Conv2d    | 3.6 K  | train
1  | pool1   | MaxPool2d | 0      | train
2  | conv2   | Conv2d    | 147 K  | train
3  | pool2   | MaxPool2d | 0      | train
4  | conv3   | Conv2d    | 147 K  | train
5  | pool3   | MaxPool2d | 0      | train
6  | flatten | Flatten   | 0      | train
7  | fc1     | Linear    | 11.1 M | train
8  | fc2     | Linear    | 16.5 K | train
9  | fc3     | Linear    | 8.3 K  | train
10 | fc4     | Linear    | 4.2 K  | train
11 | fc5     | Linear    | 325    | train
12 | softmax | Softmax   | 0      | train
-----------------------------------------------
11.4 M    Trainable params
0         Non-trainable params
11.4 M    Total params
45.615    Total estimated model params size (MB)
c:\Users\Michael\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'tra

In [ ]:
trainer.test(model, test_loader)